In [1]:
import pandas as pd
import spacy
import numpy as np
import nltk
nltk.download('stopwords')
#spacy doesnt work properly in collab, doesn't recognize nlp() function

def list_to_phrase(l):
    temp = ''
    for elm in l:
        temp += elm
        temp += ' '
    return temp[:-1]

def remove_all_blanks(l):
    while '' in l:
        l.remove('')

def get_bigrams(l):
    return list(zip(l[:-1],l[1:]))

def prob_given_parent(bigram, corpus): #calculates probability of child given parent in bigram
    corpus = flatten(corpus)
    numerator = corpus.count(bigram)
    zipped_corpus = list(zip(*corpus))
    denom = zipped_corpus[1].count(bigram[1])
    return numerator/denom

def prob_in_corpus(bigram, corpus): #calculates frequency of bigram in corpus
    corpus = flatten(corpus)
    return corpus.count(bigram)/len(corpus)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tinman\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
url = "https://raw.githubusercontent.com/CCrisanti/nlp/master/alexa?token=AFRYDDSV3K6WOA6KVPF2OZC5ZNX5C"
reviews = pd.read_csv(url, sep="\n", header=None)
reviews.head()
stop_words = list(nltk.corpus.stopwords.words('english'))
nlp = spacy.load('en')
#print(reviews.shape)

#creating chunks
reviews = list(reviews.iloc[:,0])
parsed_reviews = [list(nlp(sent).noun_chunks) for sent in reviews]
parsed_reviews = [[str(chunk).split(' ') for chunk in elm] for elm in parsed_reviews]


#removing common pronouns from chunks
for i in range(len(parsed_reviews)):
    for j in range(len(parsed_reviews[i])):
        parsed_reviews[i][j] = [word.lower() for word in parsed_reviews[i][j]]
        for word in stop_words:
            if word.lower() in parsed_reviews[i][j]:
                parsed_reviews[i][j].remove(word)
        parsed_reviews[i][j] =  list_to_phrase(parsed_reviews[i][j]) 
    if '' in parsed_reviews[i]:
        remove_all_blanks(parsed_reviews[i])

#creating bigrams
parsed_reviews_bi = []
for i in range(len(parsed_reviews)):
    parsed_reviews_bi.append(get_bigrams(parsed_reviews[i]))
    
print(parsed_reviews_bi)

[[('great gift', 'anyone'), ('anyone', 'iot devices')], [('gift', 'best buy'), ('best buy', 'echo products'), ('echo products', 'store'), ('store', 'amazon')], [('mother', 'law')], [('designed, good sound', 'everything'), ('everything', 'hd video'), ('hd video', 'answers'), ('answers', 'associated video'), ('associated video', 'text'), ('text', 'movie trailers'), ('movie trailers', 'amazon video'), ('amazon video', 'demand'), ('demand', 'amazon compatible cameras'), ('amazon compatible cameras', 'voice'), ('voice', 'message and/or video calls'), ('message and/or video calls', 'amazon show owners'), ('amazon show owners', 'alexa app holders')], [('three echo dots', 'echo show')], [('lot', 'cool stuff')], [('great looking design', 'amazon echo show')], [('product', 'anyone')], [('weeks', 'life')], [('device', 'smart home technology'), ('smart home technology', 'äò i‚äôm'), ('äò i‚äôm', 'second one'), ('second one', 'upstairs')], [('daughter', 'box'), ('box', 'box')], [('overall product',

In [6]:
import collections
newlist = [item for items in parsed_reviews_bi for item in items]
#print(newlist)
parents = []
children = []
for pair in newlist:
    parents.append(pair[0])
    children.append(pair[1])

parents_counter = collections.Counter(parents)
children_counter = collections.Counter(children)
pair_counter = collections.Counter(newlist)


#returns a tuple in the form ( p(parent|child), p(child|parent) )
def freq(parent, child):
    parent_count = parents_counter.get(parent)
    child_count = children_counter.get(child)
    pair_count = pair_counter.get((parent,child))

    return (pair_count/parent_count, pair_count/child_count)
    
print(freq('music', 'lyrics'))

110
25
7
(15.714285714285714, 3.5714285714285716)


In [ ]:
#print(reviews)
nltk.download("vader_lexicon")
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [ ]:
reviews_df = pd.read_csv(url, sep="\n", header=None)

for index, row in reviews_df.iterrows():
  print(row[0])
  polarities = sid.polarity_scores(row[0])
  print(polarities)
  print("\n")
#For now all the polarities are just printed but we could easily store them in a list or a dictionary or whatever